# Eval IoU of Segmentation model with different inpul image angle for the rotation equivariance model

In [1]:
%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils import * 
import eval_train as ev
import my_datasets as mdset
#from fcn8 import FCN8_VGG16

In [2]:
SAVE_DIR = '/data/model'
dataroot_voc = '/data/voc2012'
dataroot_sbd = '/data/sbd'


In [3]:
batch_size = 2
rotate = False
criterion_unsupervised = nn.CrossEntropyLoss(ignore_index=21)
#criterion_supervised = nn.CrossEntropyLoss(ignore_index=21) # On ignore la classe border.
Loss = 'CE' # Loss = 'KL' or 'CE' or None for L1,MSE…
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device :",device)

device : cuda:0


In [4]:
train_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='train', download=True,rotate=rotate)
val_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='val', download=True)
train_dataset_SBD = mdset.SBDataset(dataroot_sbd, image_set='train_noval',mode='segmentation',rotate=rotate)

Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar


In [5]:
dataloader_train_VOC = torch.utils.data.DataLoader(train_dataset_VOC, batch_size=batch_size,\
                                                       shuffle=True,drop_last=True)
dataloader_val = torch.utils.data.DataLoader(val_dataset_VOC, batch_size=batch_size)

## Load model

In [6]:
! ls /data/model

SVM_transfer_30_voc.pt
cnn_classif_transfer_30_voc.pt
deeplabv3_10_autosup_voc.pt
deeplabv3_30_semisup_g10_l2_voc.pt
deeplabv3_30_semisup_g50_alt_voc.pt
deeplabv3_30_semisup_voc.pt
deeplabv3_30_voc.pt
deeplabv3_50_voc.pt
deeplabv3_autosupervised_voc07.pt
deeplabv3_unsupervised_voc.pt
deeplabv3_voc.pt
dlv3_pretrained_false.pt
dlv3_voc_sbd_100.pt
dlv3_voc_sbd_30.pt
dlv3_voc_sbd_rotate_30.pt
dlv3_voc_sbd_semisup_g0.pt
dlv3_voc_sbd_semisup_g05.pt
dlv3_voc_sbd_semisup_g1.pt
fcn16.pt
fcn16_bn.pt
fcn32.pt
fcn32_bn.pt
fcn8.pt
fcn8_bn.pt
fcn_pt_pretrain_semisup_g0_L1.pt
fcn_voc_sbd30_semisup1_g05_KL_ep0.pt
fcn_voc_sbd30_semisup1_g05_KL_ep1.pt
fcn_voc_sbd30_semisup1_g05_KL_ep10.pt
fcn_voc_sbd30_semisup1_g05_KL_ep11.pt
fcn_voc_sbd30_semisup1_g05_KL_ep12.pt
fcn_voc_sbd30_semisup1_g05_KL_ep13.pt
fcn_voc_sbd30_semisup1_g05_KL_ep14.pt
fcn_voc_sbd30_semisup1_g05_KL_ep15.pt
fcn_voc_sbd30_semisup1_g05_KL_ep16.pt
fcn_voc_sbd30_semisup1_g05_KL_ep17.pt
fcn_voc_sbd30_semisup1_g05_KL_ep18.pt
fcn_voc_sbd30_se

In [7]:
def load_model(file=None,fcn=False):
    if file is None:
        if fcn is False:
            model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
        else:
            model = torchvision.models.segmentation.fcn_resnet101(pretrained=True)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [8]:
model = load_model(file='fcn_voc_sbd30_semisup1_g05_KL_ep19.pt')
model.to(device)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [9]:
model.eval()
print("EVAL ON TRAIN VOC 2012:")
m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_train_VOC,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
d_iou = ev.eval_model_all_angle(model,train=True,batch_size=batch_size,device=device,num_classes=21)
print("EVAL ON VAL VOC 2012:")
m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_val,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
d_iou = ev.eval_model_all_angle(model,train=False,batch_size=batch_size,device=device,num_classes=21)

EVAL ON TRAIN VOC 2012:


/workspace/utils.py:143: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/workspace/utils.py:145: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


Mean Pixel Accuracy between masks : 0.8948315740977913 Loss Validation : 0.32087162
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Scores for datasets rotate by 0 degrees:
   mIoU 0.5548123088165193 Accuracy 0.8466233514606978 CE Loss 0.5434119888311608
Scores for datasets rotate by 10 degrees:
   mIoU 0.5268096251539204 Accuracy 0.8413524766991626 CE Loss 0.5604919073416195
Scores for datasets rotate by 20 degrees:
   mIoU 0.4935260467460531 Accuracy 0.8317718084117438 CE Loss 0.5871318546658169